In [1]:
import pandas as pd
import numpy as np

In [2]:
df2=pd.read_excel(r"C:\Users\pc\Desktop\מדעי הנתונים\project\dict1.xlsx")

In [3]:
df=pd.read_excel("dict1.xlsx")
df.head()

,name,rate,number of reviews,starting price,cuisins,dress code,parking details,additional,Hours of operation,payment options,dining style
0,Caravaggio New York,4.7,1140 Reviews,$31 to $50,Italian,Business Casual,Nearest Parking Garages:,"Bar/Lounge, Beer, Cocktails, Delivery, Full Ba...","['Lunch', 'Daily 12:00 pm–3:00 pm', 'Dinner', ...","AMEX, Discover, MasterCard, Visa",Fine Dining
1,HAVEN Riverfront Restaurant and Bar,4.6,2803 Reviews,$31 to $50,"Contemporary American, Steak, Seafood",Smart Casual,We offer Valet and we have parking garage for ...,"Bar/Lounge, Beer, Cafe, Cocktails, Full Bar, H...","['Brunch', 'Sat 11:00 am–2:30 pm', 'Sun 11:00 ...","AMEX, Discover, MasterCard, Visa",Casual Elegant
2,The Mermaid Inn Uptown,4.7,4007 Reviews,$31 to $50,Seafood,Casual Dress,There is free street parking after 7pm Monday-...,"Beer, BYO Wine, Cafe, Corkage Fee, Full Bar, H...","['Mon–Sat 4:30 pm–10:00 pm', 'Brunch', 'Sat, S...","AMEX, Diners Club, Discover, MasterCard, Visa",Casual Dining
3,Carmine's - 91st Street - NYC,4.5,6778 Reviews,$31 to $50,"Italian, Contemporary Italian, Regional Italia...",Casual Dress,Champion Parking - 114 West 92nd St. (Between ...,"Banquet, Bar/Lounge, Beer, Cafe, Cocktails, Co...","['Mon–Thu, Sun 11:30 am–10:00 pm', 'Fri, Sat 1...","AMEX, Diners Club, Discover, MasterCard, Visa",Casual Dining
4,IL Carino Restaurant,4.7,182 Reviews,$31 to $50,Italian,Business Casual,Street Parking,"Beer, Cocktails, Full Bar, Non-Smoking, Patio/...",['Mon–Sat 4:00 pm–10:00 pm'],"AMEX, Discover, MasterCard, Visa",Elegant Dining


The number of not rated resturants is small, so we can remove them:

In [4]:
print(df[df['rate']=='No Reviews'].shape[0])
df = df[df['rate']!='No Reviews']

31


Convert 'rate' to numeric values, convert 'cuisins' to string

In [5]:
df['rate'] = pd.to_numeric(df['rate'])
df['cuisins']=df['cuisins'].astype(str)

Clean 'number of reviews' string

In [6]:
df['number of reviews']=df['number of reviews'].apply(lambda x: x.replace(' Reviews',''))

Replace 'No' with number of reviews mean

In [7]:
num_reviews = pd.to_numeric(df[df['number of reviews']!='No']['number of reviews'])
replace_style = {'No' : str(num_reviews.mean())}
df['number of reviews'] = df['number of reviews'].replace(replace_style)
df['number of reviews'] = pd.to_numeric(df['number of reviews'])

Felling 'Payment options' with mode',
Split 'Payment options' and replace it with the number of payment options available.

In [8]:
df['payment options'] = df['payment options'].fillna(df['payment options'].mode()[0])
df['payment options'] = df['payment options'].str.split(", ", expand = True).notna().sum(axis=1)

droping duplicates, fillng the null values with the most common

In [9]:
df.drop_duplicates()
df.additional = df.additional.fillna(df.additional.mode()[0]) 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4469 entries, 0 to 4499
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                4469 non-null   object 
 1   rate                4469 non-null   float64
 2   number of reviews   4469 non-null   float64
 3   starting price      4469 non-null   object 
 4   cuisins             4469 non-null   object 
 5   dress code          4469 non-null   object 
 6   parking details     4469 non-null   object 
 7   additional          4469 non-null   object 
 8   Hours of operation  4452 non-null   object 
 9   payment options     4469 non-null   int64  
 10  dining style        4469 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 419.0+ KB


In [10]:
for i in df.index:
    if(df['starting price'][i] != "$30 and under" and df['starting price'][i] != '$31 to $50'  and df['starting price'][i] != '$50 and over'):
        df['starting price'][i] = '$30 and under'

0       1140.0
1       2803.0
2       4007.0
3       6778.0
4        182.0
         ...  
4495    1234.0
4496    1423.0
4497     596.0
4498     744.0
4499    2112.0
Name: number of reviews, Length: 4469, dtype: float64

convert categorical data to numerical data

In [11]:
df['dining style'] = df['dining style'].astype('category')
replace_style = {'Casual Dining': 1, 'Casual Elegant': 2,'Fine Dining': 3,'Elegant Dining' : 4, 'View details' : 1}
df['dining style'] = df['dining style'].replace(replace_style)
replace_price = {'$30 and under': 1, '$31 to $50': 2, '$50 and over': 3}
df['starting price'] = df['starting price'].replace(replace_price)
replace_dress ={'Casual Dress':1,'Business Casual':2,'Smart Casual':3,'Resort Casual':4,'Jacket Preferred':5,'Formal Attire':6,'Jacket Required':7}
df['dress code'] = df['dress code'].replace(replace_dress)


Split 'cuisins' and get only the first variable, convert categorical data to numerical data

In [12]:
df['cuisins'] = df['cuisins'].str.split(", ", expand = True)[0]
df['cuisins'] = df['cuisins'].astype('category').cat.codes

Split 'additional' information

In [15]:
additional = df['additional'].str.split(", ", expand = True)
additional.melt()['value'].value_counts()

Patio/Outdoor Dining       5839
Wine                       4205
Full Bar                   4023
Beer                       4020
Wheelchair Access          3690
Takeout                    3521
Non-Smoking                3452
Cocktails                  3433
Bar/Lounge                 3106
Weekend Brunch             2630
Delivery                   2471
Gluten-free Options        2411
Happy Hour                 2336
Private Room               2070
Corkage Fee                2055
Banquet                    1679
Late Night                 1625
Counter Seating            1356
BYO Wine                   1227
Cafe                       1100
View                        953
Gender Neutral Restroom     948
Entertainment               769
Vegan                       559
Indoor Fireplace            539
Outdoor Smoking Area        422
Chef's Table                315
Bar Dining                  272
Dancing                     198
Beer Garden                 186
Dog Friendly                103
BYO Liqu

Adding columns from additional values

In [14]:
for col in additional.melt()['value'].value_counts().index[:1]:
    df[col] = np.where(df["additional"].str.contains(col), 1, 0)
for col in additional.melt()['value'].value_counts().index[8:11]:
    df[col] = np.where(df["additional"].str.contains(col), 1, 0)
for col in additional.melt()['value'].value_counts().index[3:4]:
    df[col] = np.where(df["additional"].str.contains(col), 1, 0)
for col in additional.melt()['value'].value_counts().index[9:10]:
    df[col] = np.where(df["additional"].str.contains(col), 1, 0)

delete coloums

In [ ]:
df.drop(columns=['name','parking details','Hours of operation','additional'],inplace=True)
df

In [ ]:
df.to_excel("nomericdata.xlsx")